In [1]:
#import statements
import pandas as pd
import numpy as np

#data visualization
import matplotlib.pyplot as ply
import seaborn as sns

#sci-kit learn
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.dummy import DummyClassifier

from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Data Cleaning and EDA

The data was sourced from the Taarifa waterpoint dashboard, which aggregates data from the Tanzania Ministry of Water. The information collected was recorded by GeoData Consultants Ltd. There are 59,400 rows and 40 columns in the "water_well_train_data.csv". 

Our target data is stored in "water_well_train_labels.csv". There are 59,400 rows and 2 columns in this csv file. The two columns in this csv file are 'id' and 'status_group'. The 'id' column aligns with the 'id' column in the "water_well_train_data.csv" file. Our target column is 'status_group' which consists of three values describing the status of a water pump: "functional", "functional needs repair", and "non-functional".

In [2]:
#import train data
#DO NOT LOOK AT TEST DATA UNTIL VALIDATION
df_train = pd.read_csv('../data/water_well_train_data.csv')

In [3]:
df_train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [4]:
# info of train data
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

The data contains 59,400 rows and 40 columns.

In [5]:
#import target information
df_label = pd.read_csv('../data/water_well_train_labels.csv')

In [6]:
df_label.shape

(59400, 2)

In [7]:
df_label.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [8]:
df_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


In [9]:
df_label['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [10]:
df_label['status_group'].value_counts(normalize = True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

There are three target classifications: functional (54%), non-functional (38%), and function needs repair (7%).

In [11]:
#combine train and label dataframes prior to cleaning to address any dropped rows
df = df_train.join(other = df_label, rsuffix = '_label')

In [12]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,69572,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,8776,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,34310,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,67743,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,19728,functional


In [13]:
#check that id columns align
df[df['id'] != df['id_label']]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

Columns with nulls:
- funder
- installer
- subvillage
- public_meeting
- scheme_management
- scheme_name
- permit

In [15]:
df.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,id_label
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475,37115.131768
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547,21453.128371
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000,18519.750000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000,37061.500000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000,55656.500000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000,74247.000000


## Dealing with Nulls

### funder

In [16]:
#who funded the well
print(df['funder'].isna().sum())
df['funder'].value_counts()

3635


Government Of Tanzania      9084
Danida                      3114
Hesawa                      2202
Rwssp                       1374
World Bank                  1349
                            ... 
Pwc                            1
Lg                             1
Malola                         1
Moses                          1
Haydom Lutheran Hospital       1
Name: funder, Length: 1897, dtype: int64

In [17]:
unique_funder = list(df['funder'].unique())
print(len(unique_funder))
unique_funder

1898


['Roman',
 'Grumeti',
 'Lottery Club',
 'Unicef',
 'Action In A',
 'Mkinga Distric Coun',
 'Dwsp',
 'Rwssp',
 'Wateraid',
 'Isingiro Ho',
 'Private',
 'Danida',
 'World Vision',
 'Lawatefuka Water Supply',
 'Biore',
 'Rudep',
 'Hesawa',
 'Twe',
 'Isf',
 'African Development Bank',
 'Government Of Tanzania',
 'Sobodo',
 'Water',
 'Private Individual',
 'Undp',
 nan,
 'Not Known',
 'Kirde',
 'Cefa',
 'Ces(gmbh)',
 'European Union',
 'Lga',
 'District Council',
 'Muwsa',
 'Dwe/norad',
 'Kkkt_makwale',
 'Sawaka',
 'Ces (gmbh)',
 'Olgilai Village Community',
 'Kkkt',
 'Roman Catholic',
 'Norad',
 'Adra',
 'Sema',
 'Piusi',
 'Dwe',
 'Rc Church',
 'Swisland/ Mount Meru Flowers',
 'Ifad',
 'Swedish',
 'Idc',
 'He',
 'Isf/tacare',
 'Jica',
 'Mzee Sh',
 'Aict',
 'Tcrs',
 'Kiuma',
 'Germany Republi',
 'Netherlands',
 'Ruthe',
 'Tulawaka Gold Mine',
 'Nethalan',
 'Tasaf',
 'Concern World Wide',
 'Wfp',
 'Lips',
 'Sida',
 'World Bank',
 'Tanza',
 '0',
 'Sw',
 'Shipo',
 'Fini Water',
 'Kanisa',
 'Ox

In [18]:
df[df['funder'] == '0']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
145,28669,50.0,2013-03-21,0,12,0,39.185710,-6.892593,Chekanao,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,28669,functional
152,60983,0.0,2013-03-16,0,-15,0,39.527114,-6.988748,Msikitini,0,...,salty,dry,dry,machine dbh,borehole,groundwater,other,other,60983,non functional
393,39749,0.0,2013-03-18,0,28,0,39.159887,-6.902548,Kwa Chambuso,0,...,unknown,unknown,unknown,machine dbh,borehole,groundwater,other,other,39749,non functional
417,15832,50.0,2013-03-22,0,30,0,39.178404,-6.938013,Ccm Kivule,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,15832,non functional
428,50233,0.0,2013-03-12,0,30,0,39.178849,-6.973206,Ofisi Ya Kata,0,...,unknown,dry,dry,shallow well,shallow well,groundwater,other,other,50233,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59237,2138,59.0,2013-03-19,0,81,0,39.119109,-6.898919,Kata,0,...,salty,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,2138,non functional
59243,3396,50.0,2013-03-16,0,-20,0,39.524021,-6.984802,Kwa Mariwala,0,...,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,3396,functional
59276,62818,50.0,2013-03-21,0,18,0,39.183790,-6.897566,Kwa Mkunduge,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,62818,functional
59351,55322,50.0,2013-03-18,0,-19,0,39.534599,-7.088183,Kwa China,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,55322,functional


In [19]:
#replace nulls in 'funder' col with "Not Known"
df['funder'].fillna(value="Not Known", inplace=True)

In [20]:
#confirm replacement of nulls
df['funder'].isna().sum()

0

### installer

In [21]:
#organization that installed the well
df['installer'].value_counts()

DWE           17402
Government     1825
RWE            1206
Commu          1060
DANIDA         1050
              ...  
Tansi             1
Laizer            1
VIFAF             1
Mwamama           1
mbeje             1
Name: installer, Length: 2145, dtype: int64

In [22]:
df['installer'].isna().sum()

3655

In [23]:
list(df['installer'].unique())

['Roman',
 'GRUMETI',
 'World vision',
 'UNICEF',
 'Artisan',
 'DWE',
 'DWSP',
 'Water Aid',
 'Private',
 'DANIDA',
 'Lawatefuka water sup',
 'WEDECO',
 'Danid',
 'TWE',
 'ISF',
 'Kilolo Star',
 'District council',
 'Water',
 'WU',
 nan,
 'Not known',
 'Central government',
 'CEFA',
 'Commu',
 'Accra',
 'World Vision',
 'LGA',
 'MUWSA',
 'KKKT _ Konde and DWE',
 'Government',
 'Olgilai village community',
 'KKKT',
 'RWE',
 'Adra /Community',
 'SEMA',
 'SHIPO',
 'HESAWA',
 'ACRA',
 'Community',
 'IFAD',
 'Sengerema Water Department',
 'HE',
 'ISF and TACARE',
 'Kokeni',
 'DA',
 'Adra',
 'ALLYS',
 'AICT',
 'KIUMA',
 'CES',
 'District Counci',
 'Ruthe',
 'Adra/Community',
 'Tulawaka Gold Mine',
 'KKT C',
 'Hesawa',
 'Water board',
 'LOCAL CONTRACT',
 'WFP',
 'LIPS',
 'TASAF',
 'World',
 '0',
 'SW',
 'Shipo',
 'Fini water',
 'Kanisa',
 'OXFARM',
 'VILLAGE COUNCIL Orpha',
 'Villagers',
 'Idara ya maji',
 'FPCT',
 'WVT',
 'Ir',
 'DANID',
 'Angli',
 'secondary school',
 'Amref',
 'JBG',
 'DAD

In [24]:
df['installer'].fillna(value='Not known', inplace=True)

In [25]:
df['installer'].isna().sum()

0

In [26]:
#check cases where the installer is not also the funder
df[df['funder'] != df['installer']]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,8776,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,34310,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,67743,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,19728,functional
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,0,...,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,9944,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,60739,functional
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,27263,functional
59397,37057,0.0,2011-04-11,Not Known,0,Not known,34.017087,-8.750434,Mashine,0,...,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,37057,functional
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,31282,functional


In [27]:
df[df['installer']=='World Bank']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
280,9474,250.0,2013-02-12,World Bank,1343,World Bank,30.017098,-4.332583,Jeshini,0,...,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,9474,functional
304,43345,0.0,2012-10-23,World Bank,0,World Bank,33.430917,-4.389084,Shule Ya Msingi,0,...,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,43345,functional
2071,28588,0.0,2012-10-23,World Bank,0,World Bank,33.436073,-4.421944,Mwanza Road,0,...,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,28588,functional
5699,74077,0.0,2012-10-18,World Bank,0,World Bank,33.435268,-4.671744,Kwa Fupe,0,...,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,74077,non functional
6327,44441,0.0,2012-10-12,World Bank,0,World Bank,33.150261,-3.705625,Kwa Maraba,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,44441,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58348,54574,0.0,2012-10-23,World Bank,0,World Bank,33.442457,-4.426592,Kilabuni,0,...,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,54574,functional
58434,54545,250.0,2013-02-12,World Bank,1311,World Bank,30.017351,-4.323850,Mission,0,...,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,54545,functional
58681,6335,250.0,2013-02-12,World Bank,1306,World Bank,30.017716,-4.308701,Kwa Thomas,0,...,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,6335,functional
58691,67819,0.0,2012-10-12,World Bank,0,World Bank,33.142605,-3.699442,Kwa Nyamizi Maswa,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,67819,non functional


In [28]:
df[(df['installer']=='World') & (df['funder']!='World Bank')]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
33726,63910,0.0,2011-04-02,Nethe,141,World,38.204463,-6.870355,Kwa Kiwele,0,...,salty,insufficient,insufficient,shallow well,shallow well,groundwater,other,other,63910,non functional


In [29]:
df[df['installer'] == 'Not known']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
34,41583,0.0,2011-02-23,Not Known,-41,Not known,39.812912,-7.889986,Msikitini Wa Ijumaa,0,...,salty,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,41583,functional
35,57355,0.0,2013-03-28,Not Known,1546,Not known,36.618699,-3.293003,Sekondari,0,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other,57355,non functional
43,19282,0.0,2013-01-15,Not Known,1642,Not known,34.967789,-4.628921,Mvae Primary,0,...,unknown,dry,dry,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,19282,non functional
47,13620,0.0,2011-07-27,Not Known,0,Not known,33.540607,-9.172905,Mahakamani,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,13620,functional
65,51072,0.0,2013-02-09,Not Known,1415,Not known,34.621598,-5.173136,Nyambi,0,...,unknown,dry,dry,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,51072,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59357,46563,0.0,2013-02-19,Not Known,1635,Not known,34.971841,-5.098362,Shabani,0,...,unknown,dry,dry,shallow well,shallow well,groundwater,hand pump,hand pump,46563,non functional
59366,55232,0.0,2013-02-02,Not Known,1541,Not known,34.765729,-5.027725,Joshoni,0,...,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,55232,functional
59370,14796,200.0,2013-01-29,Not Known,1154,Not known,30.058731,-4.902633,Village Office,0,...,unknown,unknown,unknown,unknown,other,unknown,other,other,14796,functional
59376,34716,0.0,2013-02-03,Not Known,1581,Not known,34.821039,-5.076258,Nasingo,0,...,unknown,dry,dry,shallow well,shallow well,groundwater,other,other,34716,non functional


In [30]:
df['installer'].value_counts(normalize=True)[:20]

DWE                   0.292963
Not known             0.061818
Government            0.030724
RWE                   0.020303
Commu                 0.017845
DANIDA                0.017677
KKKT                  0.015118
Hesawa                0.014141
0                     0.013081
TCRS                  0.011902
Central government    0.010471
CES                   0.010269
Community             0.009310
DANID                 0.009293
District Council      0.009276
HESAWA                0.009074
World vision          0.006869
LGA                   0.006869
WEDECO                0.006684
TASAF                 0.006667
Name: installer, dtype: float64

In [31]:
df['installer'] = df['installer'].replace(to_replace={'0':'Not known'}, value=None)

In [32]:
df['installer'].value_counts(normalize=True)[:20]

DWE                   0.292963
Not known             0.074899
Government            0.030724
RWE                   0.020303
Commu                 0.017845
DANIDA                0.017677
KKKT                  0.015118
Hesawa                0.014141
TCRS                  0.011902
Central government    0.010471
CES                   0.010269
Community             0.009310
DANID                 0.009293
District Council      0.009276
HESAWA                0.009074
World vision          0.006869
LGA                   0.006869
WEDECO                0.006684
TASAF                 0.006667
District council      0.006599
Name: installer, dtype: float64

In [33]:
df[df['installer'] == "0"]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group


### subvillage

In [34]:
#geographic location
df['subvillage'].isna().sum()

371

In [35]:
df['subvillage'].value_counts()

Madukani        508
Shuleni         506
Majengo         502
Kati            373
Mtakuja         262
               ... 
Rabwiro           1
Kwebua A          1
Urara             1
Ngorotwa A        1
Kilindijumbe      1
Name: subvillage, Length: 19287, dtype: int64

In [36]:
list(df['subvillage'].unique())

['Mnyusi B',
 'Nyamara',
 'Majengo',
 'Mahakamani',
 'Kyanyamisa',
 'Moa/Mwereme',
 'Ishinabulandi',
 'Nyawishi Center',
 'Imalauduki',
 'Mkonomre',
 'Mizugo',
 'Ngondombwito',
 'Nkilifa',
 'Omarini',
 'Mwabasabi',
 'Tunzi',
 'Kidudumo',
 'Yeriko',
 'Center',
 'Manyanya',
 'Ibabachegu',
 'Mkanivega',
 'Mkonga Juu',
 'Msasa',
 'Kitereni',
 'Shuleni',
 'Chakahaya',
 'Kiyao',
 'Merali',
 'Karume',
 'Kudipera',
 'Mosheni',
 'Lupanga A',
 'Kilombero B',
 'Afya',
 'Ndanganyika',
 'Baura',
 'Mwanzala',
 'Nyabwai B',
 'Reli B',
 'Kilunduwe',
 'Sokoni',
 'Mwarufyu',
 'Marurani Juu',
 'Isenegeja',
 'Kachulu',
 'Mpandapanda',
 'Mlandege Juu',
 'Ikanga',
 'Msaranga Street',
 'Maporomoko',
 'Isimba',
 'Kagoye B',
 'Dhobi Street',
 'Msufini',
 'Soyekiutu',
 'Ants B',
 "Izimbya 'A'",
 'Nairobi',
 'Nkaloi',
 'Kiganza Centre',
 'Ulkusare',
 'Mzimba',
 'Mfumbu',
 'Mahalule',
 'Kihanga',
 'Iponda',
 'Kisoro',
 'Mtakuja',
 'Lembuka',
 'Mapinduzi',
 'Kalimungoma',
 'Umkituri',
 'Usita',
 'Komoro',
 'Kulasi

In [37]:
df[df['subvillage'] == 'Unknown']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group


In [38]:
df['subvillage'].fillna(value='Unknown', inplace=True)

In [39]:
df['subvillage'].isna().sum()

0

### public_meeting

In [40]:
df['public_meeting'].isna().sum()

3334

In [41]:
df['public_meeting'].value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

Treat nulls as the majority case, True.

In [42]:
df['public_meeting'].fillna(value=True, inplace=True)

In [43]:
df['public_meeting'].isna().sum()

0

### scheme_management

In [44]:
#who operates the waterpoint (organization/category)
df['scheme_management'].isna().sum()

3877

In [45]:
df['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [46]:
df[df['scheme_management'] == "None"]

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
23603,23849,50.0,2013-03-18,0,-11,Not known,39.431194,-7.100783,Kwa Nyamtawa,0,...,salty,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,23849,functional


In [47]:
df[df['funder'] == '0']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group
145,28669,50.0,2013-03-21,0,12,Not known,39.185710,-6.892593,Chekanao,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,28669,functional
152,60983,0.0,2013-03-16,0,-15,Not known,39.527114,-6.988748,Msikitini,0,...,salty,dry,dry,machine dbh,borehole,groundwater,other,other,60983,non functional
393,39749,0.0,2013-03-18,0,28,Not known,39.159887,-6.902548,Kwa Chambuso,0,...,unknown,unknown,unknown,machine dbh,borehole,groundwater,other,other,39749,non functional
417,15832,50.0,2013-03-22,0,30,Not known,39.178404,-6.938013,Ccm Kivule,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,15832,non functional
428,50233,0.0,2013-03-12,0,30,Not known,39.178849,-6.973206,Ofisi Ya Kata,0,...,unknown,dry,dry,shallow well,shallow well,groundwater,other,other,50233,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59237,2138,59.0,2013-03-19,0,81,Not known,39.119109,-6.898919,Kata,0,...,salty,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,2138,non functional
59243,3396,50.0,2013-03-16,0,-20,Not known,39.524021,-6.984802,Kwa Mariwala,0,...,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,3396,functional
59276,62818,50.0,2013-03-21,0,18,Not known,39.183790,-6.897566,Kwa Mkunduge,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,62818,functional
59351,55322,50.0,2013-03-18,0,-19,Not known,39.534599,-7.088183,Kwa China,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,55322,functional


In [48]:
df['scheme_management'].fillna(value='None', inplace=True)

In [49]:
df['scheme_management'].isna().sum()

0

### scheme_name
### DROP THIS COLUMN
Potential to drop: plethora of names, not enough context to distinguish importance
28,166 nulls limits the usability of the column 

In [50]:
#who operates the waterpoint
#how is this different from scheme_management?
df['scheme_name'].value_counts()

K                             682
None                          644
Borehole                      546
Chalinze wate                 405
M                             400
                             ... 
Wisu                            1
QWIQWI                          1
Mbawala chini water suplly      1
Shirimatunda water supply       1
Uhamin                          1
Name: scheme_name, Length: 2696, dtype: int64

In [51]:
df['scheme_name'].isna().sum()

28166

In [52]:
unique_scheme = list(df['scheme_name'].unique())
print(len(unique_scheme))
unique_scheme

2697


['Roman',
 nan,
 'Nyumba ya mungu pipe scheme',
 'Zingibali',
 'BL Bondeni',
 'None',
 "wanging'ombe water supply s",
 'Makanj',
 'Kidabu',
 'Mashangwi',
 'Quick wins Program',
 'Komaka mandaka',
 'Sobodo Borehole Scheme',
 'Kitukuni water supply',
 'BL Mwakikoti',
 'Chalinze wate',
 'Mae pipeline',
 'UNDP',
 'Ngana water supplied scheme',
 'Itun',
 'Bomala',
 'Kirua kahe pumping water trust',
 'Misiwa',
 'Mtwango water supplied sche',
 'K',
 'Machumba estate pipe line',
 'SHIMASA',
 'wangama water supply scheme',
 'Shirimatunda Water Supply',
 'Laela group water Supp',
 'Makwale water supplied sche',
 "Nyang'hwale",
 'Njoro Water Supply',
 'Kirua kahe gravity water supply trust',
 'Olgilai pipe line',
 'Mabula mountains spr',
 'Mkongoro One',
 'Maambreni gravity water supply',
 'Mwando water supply',
 'M',
 'Kaisho/Isingiro w',
 "wanging'ombe supply scheme",
 'Tove Mtwango gravity Scheme',
 'Tengeru gravity water supply',
 'Kulasi water supply',
 'Malemb',
 'Tawa',
 'Loruvani gravity 

In [53]:
df['scheme_name'].fillna(value='None', inplace = True)

In [54]:
df['scheme_name'].isna().sum()

0

In [55]:
df['scheme_name'].value_counts()

None                          28810
K                               682
Borehole                        546
Chalinze wate                   405
M                               400
                              ...  
Wisu                              1
QWIQWI                            1
Mbawala chini water suplly        1
Shirimatunda water supply         1
Uhamin                            1
Name: scheme_name, Length: 2696, dtype: int64

### Permit

assume that if a permit status is unknown, there is no permit

alternatively: drop 3056 rows (~5% of data)

In [56]:
#if the waterpoint is permitted
df['permit'].value_counts()

True     38852
False    17492
Name: permit, dtype: int64

In [57]:
df['permit'].isna().sum()

3056

In [58]:
df['permit'].fillna(value=False, inplace=True)

In [59]:
df['permit'].isna().sum()

0

## Column Exploration

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 59400 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              59400 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59400 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [61]:
#total static head (amount of water available to waterpoint)
df['amount_tsh'].value_counts()

0.0         41639
500.0        3102
50.0         2472
1000.0       1488
20.0         1463
            ...  
8500.0          1
6300.0          1
220.0           1
138000.0        1
12.0            1
Name: amount_tsh, Length: 98, dtype: int64

In [62]:
df[df['amount_tsh'] == 0.0]['status_group'].value_counts()

functional                 19706
non functional             18885
functional needs repair     3048
Name: status_group, dtype: int64

In [63]:
df['date_recorded']

0        2011-03-14
1        2013-03-06
2        2013-02-25
3        2013-01-28
4        2011-07-13
            ...    
59395    2013-05-03
59396    2011-05-07
59397    2011-04-11
59398    2011-03-08
59399    2011-03-23
Name: date_recorded, Length: 59400, dtype: object

In [68]:
sorted(df['date_recorded'].tolist(), reverse = True )
#2002 to 2013

['2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-03',
 '2013-12-

'date_recorded' is stored in yy-mm-dd format

How to encode this as some measure of age in conjunction with year constructed?

Ordinal Encoding?

In [66]:
#altitude of the well
df['gps_height']

0        1390
1        1399
2         686
3         263
4           0
         ... 
59395    1210
59396    1212
59397       0
59398       0
59399     191
Name: gps_height, Length: 59400, dtype: int64

In [67]:
df['longitude'].value_counts()

0.000000     1812
37.540901       2
33.010510       2
39.093484       2
32.972719       2
             ... 
37.579803       1
33.196490       1
34.017119       1
33.788326       1
30.163579       1
Name: longitude, Length: 57516, dtype: int64

In [68]:
df['latitude'].value_counts()

-2.000000e-08    1812
-6.985842e+00       2
-3.797579e+00       2
-6.981884e+00       2
-7.104625e+00       2
                 ... 
-5.726001e+00       1
-9.646831e+00       1
-8.124530e+00       1
-2.535985e+00       1
-2.598965e+00       1
Name: latitude, Length: 57517, dtype: int64

Drop latitude/longitude since we won't be using geographic coordinates in our model.

In [69]:
#name of the waterpoint 
df['wpt_name']

0                        none
1                    Zahanati
2                 Kwa Mahundi
3        Zahanati Ya Nanyumbu
4                     Shuleni
                 ...         
59395     Area Three Namba 27
59396       Kwa Yahona Kuvala
59397                 Mashine
59398                  Mshoro
59399         Kwa Mzee Lugawa
Name: wpt_name, Length: 59400, dtype: object

In [70]:
df['wpt_name'].value_counts()

none                3563
Shuleni             1748
Zahanati             830
Msikitini            535
Kanisani             323
                    ... 
Kasharu                1
Misoke                 1
Kwa John Kamonga       1
Kwa Mzee Delenga       1
Lugula                 1
Name: wpt_name, Length: 37400, dtype: int64

'wpt_name' is probably not a useful feature for classification. Some of these are subvillage names (like Shuleni).

In [71]:
#no context
df['num_private'].value_counts()

0      58643
6         81
1         73
5         46
8         46
       ...  
180        1
213        1
23         1
55         1
94         1
Name: num_private, Length: 65, dtype: int64

Drop 'num_private'. There is no context provided so no interpretation can be made.

In [72]:
#Geographic water basin
df['basin'].value_counts()

Lake Victoria              10248
Pangani                     8940
Rufiji                      7976
Internal                    7785
Lake Tanganyika             6432
Wami / Ruvu                 5987
Lake Nyasa                  5085
Ruvuma / Southern Coast     4493
Lake Rukwa                  2454
Name: basin, dtype: int64

In [73]:
#Geographic location
df['subvillage'].value_counts()

Madukani          508
Shuleni           506
Majengo           502
Kati              373
Unknown           371
                 ... 
Bitale B Shule      1
Omurunyinya         1
Mhezi               1
Kasangali           1
Ntanga Center       1
Name: subvillage, Length: 19288, dtype: int64

In [74]:
#geographic location
df['region'].value_counts()

Iringa           5294
Shinyanga        4982
Mbeya            4639
Kilimanjaro      4379
Morogoro         4006
Arusha           3350
Kagera           3316
Mwanza           3102
Kigoma           2816
Ruvuma           2640
Pwani            2635
Tanga            2547
Dodoma           2201
Singida          2093
Mara             1969
Tabora           1959
Rukwa            1808
Mtwara           1730
Manyara          1583
Lindi            1546
Dar es Salaam     805
Name: region, dtype: int64

In [75]:
#geographic location (coded)
df['region_code'].value_counts()

11    5300
17    5011
12    4639
3     4379
5     4040
18    3324
19    3047
2     3024
16    2816
10    2640
4     2513
1     2201
13    2093
14    1979
20    1969
15    1808
6     1609
21    1583
80    1238
60    1025
90     917
7      805
99     423
9      390
24     326
8      300
40       1
Name: region_code, dtype: int64

In [76]:
#region code is type int, but best presented as a categorical feature
#convert to type string
df['region_code'] = df['region_code'].astype(str)

In [77]:
df['region_code']

0        11
1        20
2        21
3        90
4        18
         ..
59395     3
59396    11
59397    12
59398     1
59399     5
Name: region_code, Length: 59400, dtype: object

In [78]:
#geographic location (coded)
df['district_code'].value_counts()

1     12203
2     11173
3      9998
4      8999
5      4356
6      4074
7      3343
8      1043
30      995
33      874
53      745
43      505
13      391
23      293
63      195
62      109
60       63
0        23
80       12
67        6
Name: district_code, dtype: int64

In [79]:
#district code is type int, but best presented as a categorical feature
#convert to type string
df['district_code'] = df['district_code'].astype(str)

In [80]:
df['district_code']

0         5
1         2
2         4
3        63
4         1
         ..
59395     5
59396     4
59397     7
59398     4
59399     2
Name: district_code, Length: 59400, dtype: object

In [81]:
#geographic location (city?)
df['lga'].value_counts()

Njombe          2503
Arusha Rural    1252
Moshi Rural     1251
Bariadi         1177
Rungwe          1106
                ... 
Moshi Urban       79
Kigoma Urban      71
Arusha Urban      63
Lindi Urban       21
Nyamagana          1
Name: lga, Length: 125, dtype: int64

In [82]:
#geographic location (ward?)
df['ward'].value_counts()

Igosi              307
Imalinyi           252
Siha Kati          232
Mdandu             231
Nduruma            217
                  ... 
Uwanja wa Ndege      1
Mitole               1
Ifinga               1
Simbay               1
Rasbura              1
Name: ward, Length: 2092, dtype: int64

In [83]:
#population around the well
df['population'].value_counts()

0       21381
1        7025
200      1940
150      1892
250      1681
        ...  
3241        1
1960        1
1685        1
2248        1
1439        1
Name: population, Length: 1049, dtype: int64

In [84]:
df['public_meeting'].value_counts()

True     54345
False     5055
Name: public_meeting, dtype: int64

In [85]:
#drop this column
df['recorded_by'].value_counts()

GeoData Consultants Ltd    59400
Name: recorded_by, dtype: int64

In [86]:
#how do we treat 0's in construction year?
#age as an ordinal encoded variable to properly treat 0's?
df['construction_year'].value_counts()

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64

In [87]:
df[df['construction_year'] != 0]['construction_year'].describe()
#contruction years range from 1960 to 2013, with 20709 values of 0

count    38691.000000
mean      1996.814686
std         12.472045
min       1960.000000
25%       1987.000000
50%       2000.000000
75%       2008.000000
max       2013.000000
Name: construction_year, dtype: float64

In [88]:
df['extraction_type'].value_counts()

gravity                      26780
nira/tanira                   8154
other                         6430
submersible                   4764
swn 80                        3670
mono                          2865
india mark ii                 2400
afridev                       1770
ksb                           1415
other - rope pump              451
other - swn 81                 229
windmill                       117
india mark iii                  98
cemo                            90
other - play pump               85
walimi                          48
climax                          32
other - mkulima/shinyanga        2
Name: extraction_type, dtype: int64

In [89]:
df['extraction_type_group'].value_counts()

gravity            26780
nira/tanira         8154
other               6430
submersible         6179
swn 80              3670
mono                2865
india mark ii       2400
afridev             1770
rope pump            451
other handpump       364
other motorpump      122
wind-powered         117
india mark iii        98
Name: extraction_type_group, dtype: int64

In [90]:
df['extraction_type_class'].value_counts()

gravity         26780
handpump        16456
other            6430
submersible      6179
motorpump        2987
rope pump         451
wind-powered      117
Name: extraction_type_class, dtype: int64

Pick out one extraction_type column to incorporate in the model

In [91]:
#how does this differ from scheme_management
df['management'].value_counts()

vwc                 40507
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
water authority       904
other                 844
company               685
unknown               561
other - school         99
trust                  78
Name: management, dtype: int64

In [92]:
df['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
None                 3878
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
Name: scheme_management, dtype: int64

In [93]:
df['management_group'].value_counts()

user-group    52490
commercial     3638
parastatal     1768
other           943
unknown         561
Name: management_group, dtype: int64

In [94]:
df['payment'].value_counts()

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64

In [95]:
df['payment_type'].value_counts()

never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64

The 'payment' and 'payment_type' columns reflect the same information with the same values per category. Pick one payment column to incorporate.

In [96]:
df['water_quality'].value_counts()

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64

In [97]:
df['quality_group'].value_counts()

good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64

water_quality and quality_group convey the same information with some slight differences in values. water_quality is more granular in categories

In [98]:
df['quantity'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64

In [99]:
df['quantity_group'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64

Pick one of the 'quantity' columns.

In [100]:
df['source'].value_counts()

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64

In [101]:
df['source_type'].value_counts()

spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64

In [102]:
df['source_class'].value_counts()

groundwater    45794
surface        13328
unknown          278
Name: source_class, dtype: int64

In [103]:
df['waterpoint_type'].value_counts()

communal standpipe             28522
hand pump                      17488
other                           6380
communal standpipe multiple     6103
improved spring                  784
cattle trough                    116
dam                                7
Name: waterpoint_type, dtype: int64

In [104]:
df['waterpoint_type_group'].value_counts()

communal standpipe    34625
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64

waterpoint_type and waterpoint_type_group are almost identical except the communal standpipe multiple group in 'waterpoint_type' is included in 'communal standpipe' in 'waterpoint_type_group'

In [105]:
df['id'].value_counts()

2047     1
72310    1
49805    1
51852    1
62091    1
        ..
46396    1
36155    1
34106    1
38200    1
0        1
Name: id, Length: 59400, dtype: int64

In [106]:
df['id_label'].value_counts()

2047     1
72310    1
49805    1
51852    1
62091    1
        ..
46396    1
36155    1
34106    1
38200    1
0        1
Name: id_label, Length: 59400, dtype: int64

Confirmed that no duplicate id appear in the dataset

In [107]:
#remove filler values in longitude and latitude 
# dropping longitude of 0
df = df.loc[df["longitude"] != 0]
df.longitude.value_counts()

33.090347    2
32.982698    2
37.297680    2
33.010510    2
39.093484    2
            ..
37.579803    1
33.196490    1
34.017119    1
33.788326    1
35.005922    1
Name: longitude, Length: 57515, dtype: int64

In [108]:
df['latitude'].value_counts()

-2.496459    2
-6.964258    2
-6.981884    2
-7.175174    2
-7.104625    2
            ..
-5.726001    1
-9.646831    1
-8.124530    1
-2.535985    1
-2.598965    1
Name: latitude, Length: 57516, dtype: int64

In [109]:
#combine latitude and longitude into a single column

df['coordinates'] = df[['latitude', 'longitude']].values.tolist()
npcoordinates = df['coordinates'].to_list()

In [111]:
near = NearestNeighbors(n_neighbors=1)
near.fit(npcoordinates)

NearestNeighbors(n_neighbors=1)

In [112]:
#expected results, comparing npcoordinates with itself will choose self as nearest neighbor
near.kneighbors(npcoordinates, return_distance=True)

(array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 array([[    0],
        [    1],
        [    2],
        ...,
        [57585],
        [57586],
        [57587]], dtype=int64))

In [113]:
#loop over npcoordinates and find closest non-self neighbor
#this loop takes a long time to run (iterates over every row and implements NearestNeighbors)
#the code has been commented out to avoid runtime 
lst_holder = []

for i in range(len(npcoordinates)):
    near = NearestNeighbors(n_neighbors=1)
    copy_coord = npcoordinates.copy() 
    del copy_coord[i] 

    near.fit(copy_coord)
    kneigh = near.kneighbors([npcoordinates[i]], return_distance=True)[0]
    lst_holder.append(kneigh)

lst_holder

[array([[0.0016159]]),
 array([[0.00069418]]),
 array([[0.00346602]]),
 array([[0.00132884]]),
 array([[0.00540682]]),
 array([[0.00066852]]),
 array([[0.00276652]]),
 array([[0.00085363]]),
 array([[0.02749184]]),
 array([[0.01186078]]),
 array([[0.00151871]]),
 array([[0.00528396]]),
 array([[0.00018397]]),
 array([[0.00308572]]),
 array([[0.01929798]]),
 array([[0.00208952]]),
 array([[0.00310286]]),
 array([[0.00330796]]),
 array([[0.00655366]]),
 array([[0.00267281]]),
 array([[0.00662662]]),
 array([[0.00289032]]),
 array([[0.00332816]]),
 array([[0.00357005]]),
 array([[0.00161403]]),
 array([[0.00239756]]),
 array([[0.00195068]]),
 array([[0.00317831]]),
 array([[0.00259932]]),
 array([[0.00660624]]),
 array([[0.00080578]]),
 array([[0.00434898]]),
 array([[0.00250325]]),
 array([[0.00395794]]),
 array([[0.00035192]]),
 array([[0.00065916]]),
 array([[0.00489526]]),
 array([[0.00073302]]),
 array([[0.00208548]]),
 array([[0.00793315]]),
 array([[0.00182408]]),
 array([[0.001552

In [122]:
distance_list = [distance[0][0] for distance in lst_holder]
distance_list

[0.0016159025724329706,
 0.000694184653387888,
 0.0034660242807139623,
 0.0013288389239467747,
 0.0054068151509089335,
 0.0006685209274988052,
 0.00276651817246578,
 0.0008536267780451701,
 0.027491841424428166,
 0.011860777202460695,
 0.0015187142641757405,
 0.005283961514241598,
 0.0001839690683248464,
 0.003085724673804424,
 0.01929797746741712,
 0.0020895247865945076,
 0.003102858848825215,
 0.0033079638582214593,
 0.006553660594949652,
 0.0026728084065973014,
 0.00662661600010931,
 0.0028903195790779836,
 0.0033281632278741834,
 0.003570046465591521,
 0.001614033774245614,
 0.002397562944180363,
 0.0019506776106016528,
 0.003178313830728854,
 0.0025993230777452522,
 0.006606237750144125,
 0.0008057795964806796,
 0.0043489753273415754,
 0.0025032450652906512,
 0.003957939481506026,
 0.00035191832191393794,
 0.000659163737020382,
 0.004895261293221576,
 0.0007330224444714484,
 0.0020854806471425954,
 0.007933153927439936,
 0.0018240820792904562,
 0.0015527885750463504,
 7.7077231397

In [123]:
df['distance'] = distance_list

In [127]:
#engineered feature as a distance measure to the nearest other well
df['distance']

0        0.001616
1        0.000694
2        0.003466
3        0.001329
4        0.005407
           ...   
59395    0.001980
59396    0.001702
59397    0.007410
59398    0.003486
59399    0.016541
Name: distance, Length: 57588, dtype: float64

In [125]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,id_label,status_group,coordinates,distance
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,enough,spring,spring,groundwater,communal standpipe,communal standpipe,69572,functional,"[-9.85632177, 34.93809275]",0.001616
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,8776,functional,"[-2.14746569, 34.6987661]",0.000694
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,34310,functional,"[-3.82132853, 37.46066446]",0.003466
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,67743,non functional,"[-11.15529772, 38.48616088]",0.001329
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,19728,functional,"[-1.82535885, 31.13084671]",0.005407


In [ ]:
#FINAL NULLS CHECK
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df.head()

In [126]:
#SAVE CLEAN DATA OFF TO CSV FILE FOR IMPORT INTO OTHER NOTEBOOKS
#index= False
df.to_csv('../data/water_well_train_clean.csv', index_label=False)

In [ ]:
df.columns

First Model:
Columns to include in model: 18 columns
amount_tsh 
installer 
gps_height
basin
region
district_code
lga
population
public_meeting
management
permit
extraction_type
payment
water_quality
quantity
source
waterpoint_type
status_group


Columns to not include: 24
id
id_label
date recorded (age information can be an engineered feature, save for later)
funder
construction_year
longitude
latitude
wpt_name (drop)
num_private (drop)
region_code
subvillage (too many values to encode)
ward
recorded_by (drop)
scheme_name(drop)
management_group(drop)
scheme_management
extraction_type_group
extraction_type_class
payment_type
quality_group
quantity_group
source_type
source_class
waterpoint_type_group

In [ ]:
df.info()

In [ ]:
df.select_dtypes(include='number')

In [ ]:
df['installer'].value_counts()

In [ ]:
df['installer'].value_counts(normalize=True)

In [ ]:
df['installer'].value_counts(normalize=True)[:20]